UN sitio de Ebooks tiene info sobre los reviews q los usuarios hacen de sus libros een un DataFrame con formato (user_id, bookd_id, rating, timestamp). Por otro lado tenemos info en otro DataFrame que bajamos de GoodReads: (bookd_id, book_name, avg_rating). Podemos suponer que los ids de los libros son compatibles.
-A) obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks (ignorar segundo DF)
-B) Obtener un DataFrame que indique que libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

In [1]:
import pandas as pd
import random

dataFrameEbooks = pd.DataFrame(columns = ['user_id','book_id','rating','timestamp'])
dataFrameGoodReads = pd.DataFrame(columns = ['book_id','book_name','avg_rating'])
libros = ['1984','Big Data','A Clockwork Orange','Calculo II','C Programming']
fechas = ['7/6/2018','10/3/20','20/5/2019']
for id_libro in range(len(libros)):
    for i in range(3):
        dataFrameEbooks = dataFrameEbooks.append({
            'user_id' : random.randint(0,120),
            'book_id' : id_libro,
            'rating' : random.uniform(0,100),
            'timestamp' : random.choice(fechas)
        }, ignore_index = True)
    
    dataFrameGoodReads = dataFrameGoodReads.append({
        'book_id' : id_libro,
        'book_name' : libros[id_libro],
        'avg_rating' : random.uniform(0,100)
    }, ignore_index = True)
    


In [7]:
dataFrameEbooks

,user_id,book_id,rating,timestamp
0,54,0,50.471314,10/3/20
1,41,0,72.692506,7/6/2018
2,113,0,36.913082,20/5/2019
3,39,1,92.762939,7/6/2018
4,39,1,16.604221,20/5/2019
5,120,1,84.853725,20/5/2019
6,45,2,86.371300,7/6/2018
7,61,2,15.237763,20/5/2019
8,37,2,61.701930,10/3/20
9,33,3,99.765678,7/6/2018


In [2]:
dataFrameGoodReads

,book_id,book_name,avg_rating
0,0,1984,22.106551
1,1,Big Data,52.211128
2,2,A Clockwork Orange,29.645088
3,3,Calculo II,20.393611
4,4,C Programming,1.706937


In [33]:
#A) obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks (ignorar segundo DF)
newDf = dataFrameEbooks.groupby('book_id').agg({'user_id' : 'count'})
newDf.columns = ['Cantidad reviews']
newDf = newDf.reset_index()
newDf = newDf.loc[newDf['Cantidad reviews'] >= 3]
idLibros = newDf['book_id']
dataFrameEbooks2 = dataFrameEbooks.merge(newDf, on = 'book_id', how = 'inner')
dataFrameEbooks2

,user_id,book_id,rating,timestamp,Cantidad reviews
0,104,0,40.703222,10/3/20,3
1,24,0,20.129550,7/6/2018,3
2,45,0,95.424549,20/5/2019,3
3,91,1,15.451565,20/5/2019,3
4,58,1,20.883351,20/5/2019,3
5,12,1,84.665516,7/6/2018,3
6,46,2,90.349500,20/5/2019,3
7,29,2,47.984941,20/5/2019,3
8,46,2,69.226475,20/5/2019,3
9,114,3,63.032355,10/3/20,3


In [35]:
ebooksRating = dataFrameEbooks2.groupby('book_id').agg({'rating':'mean'}).nlargest(5, 'rating')
ebooksRating = ebooksRating.reset_index()

,book_id,rating
0,2,69.186972
1,3,56.625681
2,0,52.085774
3,1,40.333478
4,4,35.476132


In [47]:
#B) Obtener un DataFrame que indique que libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.
#mergeo los 2 df
dfMerged = pd.merge(ebooksRating, dataFrameGoodReads, how = 'inner', on = 'book_id')
dfMerged = dfMerged.assign(diferencia_avg_rating = abs(dfMerged['avg_rating']-dfMerged['rating']))
dfMerged = dfMerged.assign(diferencia_mayor_20pc = dfMerged['diferencia_avg_rating'] > 20)
dfMerged[dfMerged['diferencia_mayor_20pc'] == True]

,book_id,rating,book_name,avg_rating,diferencia_avg_rating,diferencia_mayor_20pc
0,2,69.186972,A Clockwork Orange,29.645088,39.541884,True
1,3,56.625681,Calculo II,20.393611,36.232070,True
2,0,52.085774,1984,22.106551,29.979222,True
4,4,35.476132,C Programming,1.706937,33.769195,True
